# LLMs

Here we'll be interacting with a server that's exposing 2 LLMs via the runnable interface.

In [10]:
from langchain.prompts.chat import ChatPromptTemplate

In [11]:
from langserve import RemoteRunnable

openai_llm = RemoteRunnable("http://localhost:8000/openai/")
anthropic = RemoteRunnable("http://localhost:8000/anthropic/")

Let's create a prompt composed of a system message and a human message.

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences.",
        ),
        ("human", "Tell me about your favorite novel"),
    ]
).format_messages()

We can use either LLM

In [4]:
anthropic.invoke(prompt)

AIMessage(content=" My favorite novel is Moby Dick by Herman Melville. The intricate plot and rich symbolism make it a complex and rewarding read. Melville's masterful prose vividly evokes the perilous life of whalers on 19th century ships.", additional_kwargs={}, example=False)

In [ ]:
openai_llm.invoke(prompt)

As with regular runnables, async invoke, batch and async batch variants are available by default

In [6]:
await openai_llm.ainvoke(prompt)

AIMessage(content='My favorite novel is "Ulysses" by James Joyce. It\'s a complex and innovative work that explores the intricacies of human consciousness and the challenges of modernity in a highly poetic and experimental manner. The prose is richly layered and rewards careful reading.', additional_kwargs={}, example=False)

In [7]:
anthropic.batch([prompt, prompt])

[AIMessage(content=" My favorite novel is Moby Dick by Herman Melville. The epic tale of Captain Ahab's obsessive quest to kill the great white whale is a profound meditation on man's struggle against nature. Melville's poetic language immerses the reader in the mysticism of the high seas.", additional_kwargs={}, example=False),
 AIMessage(content=" My favorite novel is Moby Dick by Herman Melville. The intricate details of whaling, though tedious at times, serve to heighten the symbolism and tension leading to the epic battle between Captain Ahab and the elusive white whale. Melville's sublime yet economical prose immerses the reader in a turbulent seascape teeming with meaning.", additional_kwargs={}, example=False)]

In [8]:
await anthropic.abatch([prompt, prompt])

[AIMessage(content=' Here is a concise description of my favorite novel in three sentences:\n\nMy favorite novel is Moby Dick by Herman Melville. It is the epic saga of the obsessed Captain Ahab pursuing the white whale that crippled him through the seas. The novel explores deep philosophical questions through rich symbols and metaphors.', additional_kwargs={}, example=False),
 AIMessage(content=" My favorite novel is Moby Dick by Herman Melville. The epic tale of Captain Ahab's obsessive quest for the great white whale is a masterpiece of American literature. Melville's writing beautifully evokes the mystery and danger of the high seas.", additional_kwargs={}, example=False)]

Streaming is available by default

In [5]:
for chunk in anthropic.stream(prompt):
    print(chunk.content, end="", flush=True)

 My favorite novel is Moby-Dick by Herman Melville. The epic tale of Captain Ahab's quest to find and destroy the great white whale is a masterwork of American literature. Melville's dense, philosophical prose and digressive storytelling style make the novel a uniquely challenging and rewarding read.

In [5]:
async for chunk in anthropic.astream(prompt):
    print(chunk.content, end="", flush=True)

 My favorite novel is The Art of Language by Maximo Quilana. It is a philosophical treatise on the beauty and complexity of human speech. The prose is elegant yet precise.

In [13]:
from langchain.schema.runnable import PutLocalVar, GetLocalVar

In [14]:
comedian_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a comedian that sometimes tells funny jokes and other times you just state facts that are not funny. Please either tell a joke or state fact now but only output one.",
            ),
        ]
    )
    | openai_llm
)

joke_classifier_chain = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Please determine if the joke is funny. Say `funny` if it's funny and `not funny` if not funny. Then repeat the first five words of the joke for reference...",
            ),
            ("human", "{joke}"),
        ]
    )
    | anthropic
)


chain = (
    comedian_chain
    | PutLocalVar("joke")
    | {"joke": GetLocalVar("joke")}
    | joke_classifier_chain
    | PutLocalVar("classification")
    | {"joke": GetLocalVar("joke"), "classification": GetLocalVar("classification")}
)

In [15]:
chain.invoke({})

{'joke': AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", additional_kwargs={}, example=False),
 'classification': AIMessage(content=" not funny\nWhy don't scientists trust atoms?", additional_kwargs={}, example=False)}